In [307]:
import time
import datetime
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pybit import inverse_perpetual
from binance.spot import Spot as Client

from utils import convert_to_datetime

## Short description

ema 100 среднее - желтая
ema 190 min max - белые
ema 200 среднее - синяя
ema 210 min max - красные

![](./photo_2022-05-15_16-57-56.jpg)

## BINANCE

In [184]:
API_KEY = "DnwcF34Sg2QaRcTskn9L35zudW3hXITkIpp47FXvMRzPeay49i9BmypRBod5xFlf"
SECRET_KEY = "DA1apmOA9jzvR1NoeAtdFeQRtb5f7lu8pGEKyiiT8jH9QLGRF2G6b218HmQtLbmu"
client = Client(API_KEY, SECRET_KEY)

## BYBIT

In [97]:
API_KEY = "HrZH0ZeUa1wRBrFmwx"
SECRET_KEY = "OsyhuUE6vf7gfMhHBkcYbcAKOmtPz0044QmX"
session = inverse_perpetual.HTTP(
    endpoint='https://api.bybit.com',
    api_key=API_KEY,
    api_secret=SECRET_KEY
)

## Create dataframe

In [186]:
columns = ["Date", "Open", "High", "Low", "Close", "Volume", "DateClose", "quote_asset_volume",
           "number_of_trades", "taker_buy_base", "taker_buy_quote", "ignore"]


def create_dataframe_from_binance_data(data):
    df = pd.DataFrame(data, columns=columns)
    df["Date"] = df["Date"].apply(convert_to_datetime)
    df["DateClose"] = df["DateClose"].apply(convert_to_datetime)
    df["Open"] = df["Open"].apply(lambda x: float(x))
    df["High"] = df["High"].apply(lambda x: float(x))
    df["Low"] = df["Low"].apply(lambda x: float(x))
    df["Close"] = df["Close"].apply(lambda x: float(x))
    df["Volume"] = df["Volume"].apply(lambda x: float(x))
    df.drop(["quote_asset_volume", "taker_buy_quote", "taker_buy_base", "number_of_trades", "ignore"],
            inplace=True, axis=1)
    df['diff'] = df['Close'] - df['Open']
    df.loc[df['diff'] >= 0, 'color'] = 'green'
    df.loc[df['diff'] < 0, 'color'] = 'red'

    return df

## Plot graphs to check correctnes

In [309]:
def plot_stocks(df):
    fig3 = make_subplots(specs=[[{"secondary_y": True}]])
    fig3.add_trace(go.Candlestick(x=df.index,
                                  open=df['Open'],
                                  high=df['High'],
                                  low=df['Low'],
                                  close=df['Close'],
                                  name='Price'))
    fig3.add_trace(go.Scatter(x=df.index, y=df['Close'].ewm(span=100).mean(), marker_color='yellow', name='100 MA'))
    fig3.add_trace(go.Scatter(x=df.index, y=df['Close'].ewm(span=200).mean(), marker_color='blue', name='200 Day MA'))
    fig3.add_trace(go.Scatter(x=df.index, y=df['Low'].ewm(span=190).mean(), marker_color='green', name='190 Day, short'))
    fig3.add_trace(go.Scatter(x=df.index, y=df['High'].ewm(span=190).mean(), marker_color='green', name='190 Day, long'))
    fig3.add_trace(go.Scatter(x=df.index, y=df['Low'].ewm(span=210).mean(), marker_color='red', name='210 Day, short'))
    fig3.add_trace(go.Scatter(x=df.index, y=df['High'].ewm(span=210).mean(), marker_color='red', name='210 Day, long'))
    fig3.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume', marker={'color': df['color']}), secondary_y=True)
    fig3.update_yaxes(range=[0, 700000000], secondary_y=True)
    fig3.update_yaxes(visible=False, secondary_y=True)
    fig3.update_layout(xaxis_rangeslider_visible=False)  #hide range slider
    fig3.update_layout(title={'text': 'TSLA', 'x': 0.5})
    fig3.show()

In [310]:
data = client.klines("AVAXUSDT", interval="30m", limit=1000)
df = create_dataframe_from_binance_data(data)
plot_stocks(df)

## Algorithm
1. Parse historical data
2. Parse current data every minutes
3. Find state
4. Buy
5. Sell
6. Repeat cycle for 3-5


In [313]:
token = "AVAXUSDT"
historical_data = client.klines(token, interval="30m", limit=1000)
historical_df = create_dataframe_from_binance_data(historical_data)

last_open_date = historical_df.iloc[-1]['Date'].timestamp()
prev_time = int(datetime.datetime.now().timestamp() / 60) * 60

historical_df.drop(historical_df.tail(1).index, inplace=True)

flag = False

while True:
    time.sleep(15)
    current_time = datetime.datetime.now().timestamp()

    if current_time - prev_time >= 60:
        prev_time = int(current_time / 60) * 60

        current_data = client.klines(token, interval="1m", limit=31)
        current_df = create_dataframe_from_binance_data(current_data)
        current_df = current_df[current_df['Date'].apply(lambda x: x.timestamp()) >= last_open_date]

        if current_df.iloc[-1]['Date'].timestamp() - last_open_date >= 1800:
            current_df.drop(current_df.tail(1).index, inplace=True)
            flag = True

        high = current_df['High'].max()
        low = current_df['Low'].min()
        volume = current_df['Volume'].sum()
        open_date = current_df.iloc[0]['Date']
        close_date = current_df.iloc[-1]['DateClose']
        open_cost = current_df.iloc[0]['Open']
        close_cost = current_df.iloc[-1]['Close']
        diff = close_cost - open_cost
        color = 'green' if diff >= 0 else 'red'

        copy_historical_df = historical_df.copy()
        copy_historical_df.loc[len(copy_historical_df)] = [open_date, open_cost, high, low, close_cost, volume, close_date, diff, color]

        yellow = copy_historical_df['Close'].ewm(span=100).mean().iloc[-1]
        blue = copy_historical_df['Close'].ewm(span=200).mean().iloc[-1]
        white_1 = copy_historical_df['Low'].ewm(span=190).mean().iloc[-1]
        white_2 = copy_historical_df['High'].ewm(span=190).mean().iloc[-1]
        red_1 = copy_historical_df['Low'].ewm(span=210).mean().iloc[-1]
        red_2 = copy_historical_df['High'].ewm(span=210).mean().iloc[-1]

        print(open_date, close_date)
        print(open_cost, yellow, blue, white_2, white_1, red_2, red_1)

        ### LOGIC with buy stocks

        if flag:
            last_open_date += 1800
            historical_df.drop(historical_df.head(1).index, inplace=True).reset_index(drop=True, inplace=True)
            historical_df.loc[len(historical_df)] = [open_date, open_cost, high, low, close_cost, volume, close_date, diff, color]


2022-05-19 17:00:00 2022-05-19 17:08:59
29.11 31.136804294466952 32.34814684351041 32.58065369892867 31.9192497201717 32.81807159080288 32.1394117580565
2022-05-19 17:00:00 2022-05-19 17:09:59
29.11 31.13660627466456 32.34804733650562 32.58065369892867 31.9192497201717 32.81807159080288 32.1394117580565
2022-05-19 17:00:00 2022-05-19 17:10:59
29.11 31.13621023505979 32.34784832249603 32.58065369892867 31.9192497201717 32.81807159080288 32.1394117580565
2022-05-19 17:00:00 2022-05-19 17:11:59
29.11 31.139180532095605 32.34934092756798 32.58180556228082 31.9192497201717 32.8191143210397 32.1394117580565


KeyboardInterrupt: 

In [308]:
historical_df

,Date,Open,High,Low,Close,Volume,DateClose,diff,color
0,2022-04-28 21:00:00,68.24,68.32,67.64,68.22,60784.77,2022-04-28 21:29:59,-0.02,red
1,2022-04-28 21:30:00,68.22,68.47,67.96,68.46,48744.88,2022-04-28 21:59:59,0.24,green
2,2022-04-28 22:00:00,68.46,68.53,67.94,68.36,53812.26,2022-04-28 22:29:59,-0.10,red
3,2022-04-28 22:30:00,68.36,68.55,68.14,68.22,30008.63,2022-04-28 22:59:59,-0.14,red
4,2022-04-28 23:00:00,68.22,68.37,68.02,68.04,26728.32,2022-04-28 23:29:59,-0.18,red
...,...,...,...,...,...,...,...,...,...
994,2022-05-19 14:00:00,28.14,28.27,27.70,27.78,107719.45,2022-05-19 14:29:59,-0.36,red
995,2022-05-19 14:30:00,27.78,28.18,27.51,28.16,145329.00,2022-05-19 14:59:59,0.38,green
996,2022-05-19 15:00:00,28.16,28.44,27.86,28.35,132702.16,2022-05-19 15:29:59,0.19,green
997,2022-05-19 15:30:00,28.35,29.42,28.14,29.27,186105.73,2022-05-19 15:59:59,0.92,green


In [288]:
copy_historical_data.loc[len(copy_historical_data)] = list(copy_historical_data.iloc[0])
copy_historical_data

,Date,Open,High,Low,Close,Volume,DateClose,diff,color
0,2022-04-28 18:30:00,67.81,68.45,67.63,68.32,36957.48,2022-04-28 18:59:59,0.51,green
1,2022-04-28 19:00:00,68.33,69.29,67.82,68.30,112809.84,2022-04-28 19:29:59,-0.03,red
2,2022-04-28 19:30:00,68.32,68.45,67.25,67.76,64879.97,2022-04-28 19:59:59,-0.56,red
3,2022-04-28 20:00:00,67.76,67.89,67.21,67.61,64618.09,2022-04-28 20:29:59,-0.15,red
4,2022-04-28 20:30:00,67.61,68.48,67.53,68.22,69693.20,2022-04-28 20:59:59,0.61,green
...,...,...,...,...,...,...,...,...,...
996,2022-05-19 12:30:00,29.19,29.22,28.34,28.36,107620.72,2022-05-19 12:59:59,-0.83,red
997,2022-05-19 13:00:00,28.35,28.53,28.19,28.24,99883.28,2022-05-19 13:29:59,-0.11,red
998,2022-05-19 13:30:00,28.24,28.26,27.73,28.15,131162.07,2022-05-19 13:59:59,-0.09,red
999,2022-05-19 14:00:00,28.14,28.27,27.89,27.93,97942.24,2022-05-19 14:29:59,-0.21,red


IndexError: single positional indexer is out-of-bounds

In [267]:
curr.loc[29, ['High', 'Low']] = [1, 2]

In [271]:
historical_data = client.klines("AVAXUSDT", interval="1m", limit=30)
create_dataframe_from_binance_data(historical_data)

,Date,Open,High,Low,Close,Volume,DateClose,diff,color
0,2022-05-19 15:04:00,28.04,28.04,27.97,27.99,1630.14,2022-05-19 15:04:59,-0.05,red
1,2022-05-19 15:05:00,28.00,28.00,27.93,27.93,1378.06,2022-05-19 15:05:59,-0.07,red
2,2022-05-19 15:06:00,27.92,28.00,27.91,27.93,2709.58,2022-05-19 15:06:59,0.01,green
3,2022-05-19 15:07:00,27.93,27.95,27.92,27.94,1163.37,2022-05-19 15:07:59,0.01,green
4,2022-05-19 15:08:00,27.94,28.03,27.93,28.02,2027.18,2022-05-19 15:08:59,0.08,green
5,2022-05-19 15:09:00,28.01,28.01,27.91,27.93,3084.27,2022-05-19 15:09:59,-0.08,red
6,2022-05-19 15:10:00,27.93,27.99,27.91,27.91,3013.28,2022-05-19 15:10:59,-0.02,red
7,2022-05-19 15:11:00,27.91,27.92,27.86,27.87,7786.69,2022-05-19 15:11:59,-0.04,red
8,2022-05-19 15:12:00,27.87,28.02,27.87,28.01,2983.14,2022-05-19 15:12:59,0.14,green
9,2022-05-19 15:13:00,28.01,28.03,27.96,27.97,1806.47,2022-05-19 15:13:59,-0.04,red


In [253]:
datetime.fromtimestamp(curr.iloc[-1]['Date'].timestamp())

datetime.datetime(2022, 5, 19, 18, 6)

In [249]:
curr.iloc[22]['Date'].timestamp() - last_open_date

1740.0

In [205]:
historical_data = client.klines("AVAXUSDT", interval="30m", limit=1000)
historical_df = create_dataframe_from_binance_data(historical_data)
last_open_date = historical_df.iloc[-1]['Date']
close_open_date = historical_df.iloc[-1]['DateClose']

In [212]:
datetime

datetime.datetime

In [196]:
curr[curr['Date'] >= last_open_date]

,Date,Open,High,Low,Close,Volume,DateClose
0,2022-05-19 13:37:00,28.17,28.17,28.07,28.07,2763.35,2022-05-19 13:37:59
1,2022-05-19 13:38:00,28.08,28.11,27.94,28.03,14781.72,2022-05-19 13:38:59
2,2022-05-19 13:39:00,28.03,28.09,27.94,28.01,4202.28,2022-05-19 13:39:59
3,2022-05-19 13:40:00,28.01,28.03,27.94,27.94,5875.07,2022-05-19 13:40:59
4,2022-05-19 13:41:00,27.94,27.94,27.83,27.89,6494.09,2022-05-19 13:41:59
5,2022-05-19 13:42:00,27.89,27.94,27.85,27.92,4345.80,2022-05-19 13:42:59
6,2022-05-19 13:43:00,27.92,27.99,27.92,27.97,2035.70,2022-05-19 13:43:59
7,2022-05-19 13:44:00,27.97,27.99,27.93,27.94,3005.36,2022-05-19 13:44:59
8,2022-05-19 13:45:00,27.95,28.02,27.94,28.00,3049.76,2022-05-19 13:45:59
9,2022-05-19 13:46:00,28.00,28.09,27.99,27.99,5219.67,2022-05-19 13:46:59


In [202]:
for row in curr.iterrows():
    print(row)


def aggregate_current(dataframe: pd.DataFrame):
    pass

(0, Date         2022-05-19 13:37:00
Open                       28.17
High                       28.17
Low                        28.07
Close                      28.07
Volume                   2763.35
DateClose    2022-05-19 13:37:59
Name: 0, dtype: object)
(1, Date         2022-05-19 13:38:00
Open                       28.08
High                       28.11
Low                        27.94
Close                      28.03
Volume                  14781.72
DateClose    2022-05-19 13:38:59
Name: 1, dtype: object)
(2, Date         2022-05-19 13:39:00
Open                       28.03
High                       28.09
Low                        27.94
Close                      28.01
Volume                   4202.28
DateClose    2022-05-19 13:39:59
Name: 2, dtype: object)
(3, Date         2022-05-19 13:40:00
Open                       28.01
High                       28.03
Low                        27.94
Close                      27.94
Volume                   5875.07
DateClose    2022-05-